<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# FAST-OAD-GA Tutorial

FAST-OAD-GA is an add-on package for [FAST-OAD framework](https://github.com/fast-aircraft-design/FAST-OAD) for performing rapid Overall Aircraft Design in the category General Aviation (GA). The computational core of FAST-OAD being based on the  [OpenMDAO framework](https://openmdao.org/).

## 1. Setting up and analyzing the initial problem

To organize our work, we propose to use two user folders `data/` and `workdir/`. For instance, in `data/` we store a XML file which describes the Beechcraft Duchess. In `workdir/`, we store files generated or modified by FAST-OAD.

In [4]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path
DATA_FOLDER_PATH = 'data'
WORK_FOLDER_PATH = 'workdir'

# Remove work folder
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)

# Define files
CONFIGURATION_FILE = pth.join(WORK_FOLDER_PATH, 'oad_process.yml')
SOURCE_FILE = pth.join(DATA_FOLDER_PATH, 'beechcraft_76.xml')

# For having log messages on screen
logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

# For using all screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

After defining a configuration file name, we can ask FAST-GA to generate a default configuration file based on the default OAD model implemented in the framework:

In [ ]:
api_cs23.generate_configuration_file(CONFIGURATION_FILE, overwrite=True)

You can now checkout the generated [configuration file](./workdir/oad_process.toml). In this configuration file, we have specified an input file name 'problem_inputs.xml'. We can ask FAST-OAD to generate the inputs of the default model with the CeRAS parameters as default values:

In [6]:
api_cs25.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

INFO    : Problem inputs written in D:\a.reysset\Documents\Github_new\FAST-GA\src\fastga\notebooks\tutorial\results\oad_process_inputs.xml


'D:\\a.reysset\\Documents\\Github_new\\FAST-GA\\src\\fastga\\notebooks\\tutorial\\results\\oad_process_inputs.xml'

You can now checkout the generated [input file](./workdir/problem_inputs.xml). As shown previously in the user file architecture, the values in this file can be modified by the user and will be considered by FAST-OAD when executing a computational process.

A useful feature that FAST-OAD provides is to list the outputs of the model defined in the configuration file:

In [7]:
api_cs25.list_variables(CONFIGURATION_FILE)

NAME,I/O,DESCRIPTION
data:TLAR:NPAX_design,IN,
data:TLAR:luggage_mass_design,IN,
data:TLAR:range,IN,top-level requirement: design range
data:TLAR:v_approach,IN,
data:TLAR:v_cruise,IN,
data:TLAR:v_limit,IN,
data:TLAR:v_max_sl,IN,
data:aerodynamics:horizontal_tail:low_speed:root:CL_max_2D,IN,
data:aerodynamics:horizontal_tail:low_speed:root:CL_min_2D,IN,
data:aerodynamics:horizontal_tail:low_speed:tip:CL_max_2D,IN,


Another useful feature is to list the modules of the model defined in the configuration file:

In [8]:
api_cs25.list_modules(CONFIGURATION_FILE)

AVAILABLE MODULE IDENTIFIERS,MODULE PATH
fastga.aerodynamics.highspeed.legacy,D:\a.reysset\Documents\Github_new\FAST-GA\src\fastga\models\aerodynamics\aerodynamics_high_speed.py
fastga.aerodynamics.lowspeed.legacy,D:\a.reysset\Documents\Github_new\FAST-GA\src\fastga\models\aerodynamics\aerodynamics_low_speed.py
fastga.geometry.alternate,D:\a.reysset\Documents\Github_new\FAST-GA\src\fastga\models\geometry\geometry.py
fastga.geometry.legacy,D:\a.reysset\Documents\Github_new\FAST-GA\src\fastga\models\geometry\geometry.py
fastga.handling_qualities.static_margin,D:\a.reysset\Documents\Github_new\FAST-GA\src\fastga\models\handling_qualities\compute_static_margin.py
fastga.handling_qualities.tail_sizing,D:\a.reysset\Documents\Github_new\FAST-GA\src\fastga\models\handling_qualities\tail_sizing\update_tail_areas.py
fastga.loads.legacy,D:\a.reysset\Documents\Github_new\FAST-GA\src\fastga\models\load_analysis\loads.py
fastga.loop.mtow,D:\a.reysset\Documents\Github_new\FAST-GA\src\fastga\models\weight\mass_breakdown\update_mtow.py
fastga.loop.wing_area,D:\a.reysset\Documents\Github_new\FAST-GA\src\fastga\models\loops\update_wing_area.py
fastga.loop.wing_position,D:\a.reysset\Documents\Github_new\FAST-GA\src\fastga\models\loops\update_wing_position.py


Another useful feature is the [N2 diagram](http://openmdao.org/twodocs/versions/latest/basic_guide/make_n2.html) visualization available in OpenMDAO to see the structure of the model:

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, 'n2.html')
api_cs25.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width='100%', height='500px')

Alternatively, you can create a [WhatsOpt](https://github.com/OneraHub/WhatsOpt-Doc#whatsopt-documentation) account to generate the XDSM of the problem. If your account is created, you may uncomment next lines and run them (this should take ~ 1 min):

In [ ]:
XDSM_FILE = pth.join(WORK_FOLDER_PATH, 'xdsm.html')
api_cs25.write_xdsm(CONFIGURATION_FILE, XDSM_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=XDSM_FILE, width='100%', height='500px')

## 2. Running your first MDA

### Beechcraft
Here we run an MDA, that is solving the multidisciplinary couplings using the different nested solvers in the model, without running the optimization problem even if it is defined in the configuration file.

In [9]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

NL: NLBGS 1 ; 7822533.67 1
NL: NLBGS 2 ; 2026480.65 0.259056814
NL: NLBGS 3 ; 2867971.26 0.366629456
NL: NLBGS 4 ; 1950213.37 0.249307123
NL: NLBGS 5 ; 163600.073 0.0209139493
NL: NLBGS 6 ; 320516.094 0.0409734374
NL: NLBGS 7 ; 57635.6654 0.00736790251
NL: NLBGS 8 ; 32083.3323 0.00410139907
NL: NLBGS 9 ; 33422.4903 0.00427259143
NL: NLBGS 10 ; 17634.0248 0.00225426001
NL: NLBGS 11 ; 8022.8232 0.00102560418
NL: NLBGS 12 ; 4557.43826 0.000582603854
NL: NLBGS 13 ; 2491.76683 0.000318537054
NL: NLBGS 14 ; 1141.68308 0.000145947992
NL: NLBGS 15 ; 557.121413 7.12200723e-05
NL: NLBGS 16 ; 267.870943 3.42435015e-05
NL: NLBGS 17 ; 115.850883 1.48098925e-05


INFO    : Computation finished after 146.82 seconds
INFO    : Problem outputs written in D:\a.reysset\Documents\Github_new\FAST-GA\src\fastga\notebooks\tutorial\results\oad_process_outputs.xml


NL: NLBGS 18 ; 49.080764 6.27427967e-06
NL: NLBGS Converged


Let's save these results. We will use them in the next workbook, that shows some post-processing utilities.

In [12]:
OUTPUT_FILE = pth.join('results', 'oad_process_outputs.xml')
Beechcraft_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_800nm_mda.xml')
shutil.copy(OUTPUT_FILE, Beechcraft_OUTPUT_FILE)

'workdir\\problem_outputs_Beechcraft_800nm_mda.xml'

The `variable-viewer` provides a way to inspect the content of the XML file. The dropdown lists above the table allow to filter the displayed variable.

In [13]:
api_cs25.variable_viewer(OUTPUT_FILE)

In particular, you may inspect the `data:handling_qualities:static_margin` variable.
You will see that its value is slightly negative, which means that the current aircraft configuration is estimated as unstable. We will see later how we can correct this point.

### Beechcraft for 1000 nm range
Here we run an MDA but we change one of the Top Level Aircraft Requirement (TLAR): the range. We choose a 1000 nm range instead of 800 nm for the Beechcraft. For that we use the `VariableViewer` tool on the input file to change the range (do not forget to save!). Just like this:
![variable_viewer](./img/variable_viewer_change_range.gif)

In [ ]:
INPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs.xml')
api_cs25.variable_viewer(INPUT_FILE)

Now that the range has been changed, we run again the MDA.

In [ ]:
eval_problem = api_cs25.evaluate_problem(CONFIGURATION_FILE, overwrite=True)

Let's save again these new results, for post-processing them in next notebook.

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
Beechcraft_1000nm_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_1000nm_mda.xml')
shutil.copy(OUTPUT_FILE, Beechcraft_1000nm_OUTPUT_FILE)

## 3. Running your first MDO

## Beechcraft 800 nm optimization

As seen earlier, the current aircraft configuration is estimated with a fixed rear length (and thus fuselage length). We will unlock this parameter using a simple optimization problem that will move horizontal tail position in order to minimize the mission consumption. 

The default configuration file defines this optimization problem that aims at:
- minimizing the fuel consumption for the mission (objective),
- with respect to the rear fusellage length (design variables): by extension the horizontal tail to wing distance,
- subject to no additional constraint.

*(This run should take approximately 20 minutes)*

In [ ]:
# Set back the inputs from the reference Beechcraft 800 nm
api_cs25.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

To visualize and edit the optimization problem definition (present in the configuration file .toml) you can use the `optimization_viewer` tool. If design variables or constraints have active bounds they are yellow whereas they are red if they are violated. Modifiying the `Initial Value` will modify the input file defined in the configuration file .toml whereas `Value` corresponds to the value found in the output file defined in the configuration file (here it is the 800 nm MDA run).

In [ ]:
api_cs25.optimization_viewer(CONFIGURATION_FILE)

In [ ]:
optim_problem = api_cs25.optimize_problem(CONFIGURATION_FILE, overwrite=True)

Let's save these results:

In [ ]:
OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
CeRAS_OPT_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs_Beechcraft_800nm_mdo.xml')
shutil.copy(OUTPUT_FILE, CeRAS_OPT_OUTPUT_FILE)

The `optimizer_viewer` offers a convenient summary of the optimization result:

In [ ]:
api_cs25.optimization_viewer(CONFIGURATION_FILE)

You can use the `VariableViewer` tool to see the optimization results for all variables of the system by loading the .xml output file:

In [ ]:
RESULT_FILE = pth.join(WORK_FOLDER_PATH, 'problem_outputs.xml')
api_cs25.variable_viewer(RESULT_FILE)